# Census aggregation race data
By: Sandhya Kambhampati

In [ ]:
#load libraries
import pandas as pd

In [ ]:
# load tract to neighborhood lookup
shapes = pd.read_csv("../data/output/la_tract_to_neighborhood_portions.csv",dtype={"tract": str})

In [ ]:
len(shapes)

In [ ]:
shapes.head(1)

In [ ]:
# bring race data in for LA 
race = pd.read_csv("../data/input/acs5_2022_race_la_tracts.csv",dtype={"tract": str})

In [ ]:
race.universe.sum()

In [ ]:
race.head(1)

In [ ]:
race.columns

In [ ]:
col = ['universe','white_alone', 'black_alone','american_indian_and_alaska_native', 'asian_alone',
       'native_hawaiian_and_pacific_islander','some_other_race','two_or_more_races',
       'latino_alone']

In [ ]:
n = pd.merge(shapes,race,on="tract",how="left")

In [ ]:
len(shapes)

In [ ]:
n.universe.sum()

In [ ]:
n.tract.nunique()

In [ ]:
race.tract.nunique()

In [ ]:
shapes.tract.nunique()

In [ ]:
assert len(n) == len(shapes)

In [ ]:
def by_percent(row):
    for c in col:
        row[c] = row[c]*row["tract_percent"]
    return row
    

In [ ]:
race_cleaned = n.apply(by_percent, axis=1)

In [ ]:
race_cleaned = race_cleaned.groupby(["neighborhood"]).sum().reset_index()

In [ ]:
race_cleaned[["neighborhood","universe"]]

In [ ]:
race_cleaned

In [ ]:
race_cleaned.universe.sum()

In [ ]:
# calculate percentages for each race group         
race_cleaned["white_percent"] = race_cleaned.white_alone/race_cleaned.universe
race_cleaned["black_percent"] = race_cleaned.black_alone/race_cleaned.universe
race_cleaned["american_indian_percent"] = race_cleaned.american_indian_and_alaska_native/race_cleaned.universe
race_cleaned["asian_percent"] = race_cleaned.asian_alone/race_cleaned.universe
race_cleaned["native_hawaiian_percent"] = race_cleaned.native_hawaiian_and_pacific_islander/race_cleaned.universe
race_cleaned["other_percent"] = race_cleaned.some_other_race/race_cleaned.universe
race_cleaned["two_or_more_races_percent"] = race_cleaned.two_or_more_races/race_cleaned.universe
race_cleaned["latino_percent"] = race_cleaned.latino_alone/race_cleaned.universe
race_cleaned["non_latino_percent"]= 1-race_cleaned.latino_percent

In [ ]:
race_cleaned["nonwhite_percent"]=(1-race_cleaned.white_percent)

In [ ]:
race_cleaned["majority_nonwhite"] = race_cleaned.nonwhite_percent > 0.5
race_cleaned["majority_white"] = race_cleaned.white_percent  > 0.5
race_cleaned["majority_black"]= race_cleaned.black_percent > 0.5
race_cleaned["majority_asian"]= race_cleaned.asian_percent > 0.5
race_cleaned["majority_latino"] = race_cleaned.latino_percent > 0.5

In [ ]:
# add in diversity index from https://www.usatoday.com/story/news/nation/2014/10/21/diversity-index-data-how-we-did-report/17432103/ & https://github.com/censusreporter/nicar16-diversity-index/blob/master/Walkthrough.ipynb
race_cleaned["diversity_index"]= 1-( 
              (race_cleaned.white_percent **2 + race_cleaned.black_percent**2 + race_cleaned.american_indian_percent**2 + race_cleaned.asian_percent**2 + race_cleaned.native_hawaiian_percent**2 ) * 
              (race_cleaned.latino_percent**2 + race_cleaned.non_latino_percent**2)
           )

In [ ]:
len(race_cleaned)

In [ ]:
list(race_cleaned)

In [ ]:
# cut the df 
race_neighborhoods = race_cleaned[[
"neighborhood",
"universe",                           
'white_alone',
'black_alone',
'american_indian_and_alaska_native', 
'asian_alone',
'native_hawaiian_and_pacific_islander',
'some_other_race',
'two_or_more_races',
'latino_alone',
'white_percent',
'black_percent',
'american_indian_percent',
'asian_percent',
'native_hawaiian_percent',
'other_percent',
'two_or_more_races_percent',
'latino_percent',
'nonwhite_percent',
'majority_nonwhite',
'majority_white',
'majority_black',
'majority_asian',
'majority_latino',
'non_latino_percent',
'diversity_index']]

In [ ]:
race_neighborhoods.rename(columns={"universe": "total_population"},inplace=True)

In [ ]:
race_neighborhoods.to_csv("../data/output/race_neighborhoods.csv", index=False)

## Get top neighborhoods

In [ ]:
len(race_neighborhoods)

In [ ]:
race_neighborhoods.columns

### Plurality Black neighborhoods

In [ ]:
race_neighborhoods.loc[
    (race_neighborhoods["black_percent"] > race_neighborhoods["white_percent"]) & 
    (race_neighborhoods["black_percent"] > race_neighborhoods["latino_percent"]) &
    (race_neighborhoods["black_percent"] > race_neighborhoods["asian_percent"])
]

In [ ]:
race_neighborhoods.loc[
    (race_neighborhoods["black_percent"] > race_neighborhoods["white_percent"]) & 
    (race_neighborhoods["black_percent"] > race_neighborhoods["latino_percent"]) &
    (race_neighborhoods["black_percent"] > race_neighborhoods["asian_percent"])
]["neighborhood"].tolist()

### Majority Latino neighborhoods

In [ ]:
len(race_neighborhoods.loc[race_neighborhoods["majority_latino"] == True])

In [ ]:
race_neighborhoods.loc[race_neighborhoods["majority_latino"] == True]["neighborhood"].tolist()

### Majority white neighborhoods

In [ ]:
len(race_neighborhoods.loc[race_neighborhoods["majority_white"] == True])

In [ ]:
race_neighborhoods.loc[race_neighborhoods["majority_white"] == True]["neighborhood"].tolist()